In [111]:
import pandas as pd
import numpy as np
import folium
import geopandas as gpd
from tqdm import tqdm
import shapely.wkt
from pyproj import Geod
from collections import defaultdict

from shapely.geometry import Point,MultiPolygon

In [92]:
def get_gdf(locs):
    df = gpd.GeoDataFrame({'geometry': locs}, crs='EPSG:4326')
    df = df.to_crs('EPSG:2263')
    return df

In [112]:
geod = Geod(ellps="WGS84")

In [3]:
build = pd.read_csv('building.csv')
build.head()

In [80]:
build.head()

,the_geom,CNSTRCT_YR,BIN,NAME,LSTMODDATE,LSTSTATYPE,DOITT_ID,HEIGHTROOF,FEAT_CODE,GROUNDELEV,SHAPE_AREA,SHAPE_LEN,BASE_BBL,MPLUTO_BBL,GEOMSOURCE
0,MULTIPOLYGON (((-73.96664570466969 40.62599676...,1925.0,3170958,NaN,08/22/2017 12:00:00 AM +0000,Constructed,96807,29.749853,2100.0,40.0,0,0,3.065220e+09,3.065220e+09,Photogramm
1,MULTIPOLYGON (((-74.16790202462265 40.63936048...,1965.0,5028452,NaN,08/22/2017 12:00:00 AM +0000,Constructed,326368,22.630000,2100.0,39.0,0,0,5.012640e+09,5.012640e+09,Photogramm
2,MULTIPOLYGON (((-74.19510813278613 40.55610681...,1970.0,5078368,NaN,08/22/2017 12:00:00 AM +0000,Constructed,746627,35.760000,2100.0,51.0,0,0,5.060190e+09,5.060190e+09,Photogramm
3,MULTIPOLYGON (((-73.96113466505085 40.57743931...,1928.0,3245111,NaN,08/22/2017 12:00:00 AM +0000,Constructed,786626,37.500000,2100.0,6.0,0,0,3.086910e+09,3.086910e+09,Photogramm
4,MULTIPOLYGON (((-73.75421559146167 40.75591276...,1950.0,4161096,NaN,08/22/2017 12:00:00 AM +0000,Constructed,746409,18.015113,2100.0,93.0,0,0,4.075020e+09,4.075020e+09,Photogramm


In [117]:
build['area'] = build.the_geom.apply(lambda x: abs(geod.geometry_area_perimeter(shapely.wkt.loads(x))[0]))

In [46]:
temp = pd.read_csv('Hyperlocal_Temperature_Monitoring.csv')
temp.head()

,Sensor.ID,AirTemp,Day,Hour,Latitude,Longitude,Year,Install.Type,Borough,ntacode
0,Bk-BR_01,71.189000,06/15/2018,1,40.666205,-73.91691,2018,Street Tree,Brooklyn,BK81
1,Bk-BR_01,70.243333,06/15/2018,2,40.666205,-73.91691,2018,Street Tree,Brooklyn,BK81
2,Bk-BR_01,69.392667,06/15/2018,3,40.666205,-73.91691,2018,Street Tree,Brooklyn,BK81
3,Bk-BR_01,68.263167,06/15/2018,4,40.666205,-73.91691,2018,Street Tree,Brooklyn,BK81
4,Bk-BR_01,67.114000,06/15/2018,5,40.666205,-73.91691,2018,Street Tree,Brooklyn,BK81


In [93]:
unique_locations = temp.groupby(['Latitude','Longitude']).count().reset_index(0).reset_index(0)[['Latitude','Longitude']]
temp_locs = [Point(row['Latitude'],row['Longitude']) for _,row in unique_locations.iterrows()]

temp_coord = get_gdf(temp_locs)

In [128]:
geometry = build.the_geom.apply(lambda x: shapely.wkt.loads(x).centroid)
build_coords = gpd.GeoDataFrame({'geometry': geometry.apply(lambda row: Point(row.y,row.x)),'bin': build['BIN'], 'height': build['HEIGHTROOF'],'area':build['area']}, crs='EPSG:4326')
build_coords = build_coords.to_crs('EPSG:2263')

In [135]:
num_buildings100 = defaultdict(set)
for i,_ in tqdm(temp_coord.iterrows(),total=temp_coord.shape[0]):
    out = build_coords.geometry.distance(temp_coord.iloc[[i]*build.shape[0]].reset_index(0).drop(columns='index')) / 3.2808
    for _,row in build_coords[out<=500].iterrows():
        num_buildings100[i].add((row['height'],row['area']))

100%|██████████| 435/435 [1:07:38<00:00,  9.33s/it]


In [146]:
n = temp_coord.shape[0]
num_build500, mean_fa_ratio = [0] * n,[0] * n
for i,v in num_buildings100.items():
    num_build500[i] = (len(v))
    mean_fa_ratio[i] = np.mean([(height*3.2808)/fa for height, fa in v])        

In [147]:
unique_locations['num_build500'] = num_build500
unique_locations['mean_fa_ratio'] = mean_fa_ratio

In [149]:
unique_locations.to_csv('temp2.csv',index=False)

In [150]:
unique_locations

,Latitude,Longitude,num_build500,mean_fa_ratio
0,40.646738,-73.951234,31,1.179760
1,40.646877,-73.946154,17,0.865508
2,40.647370,-73.945400,18,0.748947
3,40.647410,-73.948200,30,0.815587
4,40.647446,-73.951205,25,1.060397
...,...,...,...,...
430,40.883276,-73.849236,10,0.824014
431,40.883292,-73.845751,3,1.208231
432,40.883486,-73.847269,19,1.502979
433,40.884042,-73.846541,2,0.138961


In [ ]:
un